# Train a Hypergraph Message Passing Neural Network (HMPNN)

In this notebook, we will create and train a Hypergraph Message Passing Neural Network in the hypergraph domain. This method is introduced in the paper [Message Passing Neural Networks for
Hypergraphs](https://arxiv.org/abs/2203.16995) by Heydari et Livi 2022. We will use a benchmark dataset, Cora, a collection of 2708 academic papers and 5429 citation relations, to do the task of node classification. There are 7 category labels, namely `Case_Based`, `Genetic_Algorithms`, `Neural_Networks`, `Probabilistic_Methods`, `Reinforcement_Learning`, `Rule_Learning` and `Theory`.

Each document is initially represented as a binary vector of length 1433, standing for a unique subset of the words within the papers, in which a value of 1 means the presence of its corresponding word in the paper.

In [1]:
import torch
import torch_geometric.datasets as geom_datasets
from sklearn.metrics import accuracy_score

from topomodelx.nn.hypergraph.hmpnn import HMPNN

torch.manual_seed(0)

/home/intel500g/thomas/miniconda3/envs/topox/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


If GPU's are available, we will make use of them. Otherwise, this will run on CPU.

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


# Pre-processing

Here we download the dataset. It contains initial representation of nodes, the adjacency information, category labels and train-val-test masks.

In [3]:
dataset = geom_datasets.Planetoid(root="tmp/", name="cora")[0]

In [4]:
dataset.num_nodes

2708

Below, we construct the incidence matrix ($B_1$) which is of shape $n_\text{nodes} \times n_\text{edges}$.

In [5]:
dataset["incidence_1"] = torch.sparse_coo_tensor(
    dataset["edge_index"], torch.ones(dataset["edge_index"].shape[1]), dtype=torch.long
)
dataset = dataset.to(device)

In [6]:
dataset.node_stores[0]['train_mask']

tensor([ True,  True,  True,  ..., False, False, False], device='cuda:0')

In [7]:
x_0s = dataset["x"]
y = dataset["y"]
incidence_1 = dataset["incidence_1"]

# Train the Neural Network

We then specify the hyperparameters and construct the model, the loss and optimizer.

In [8]:
class Network(torch.nn.Module):
    """Network class that initializes the base model and readout layer.

    Base model parameters:
    ----------
    Reqired:
    in_channels : int
        Dimension of the input features.
    hidden_channels : int
        Dimension of the hidden features.

    Optitional:
    **kwargs : dict
        Additional arguments for the base model.

    Readout layer parameters:
    ----------
    out_channels : int
        Dimension of the output features.
    task_level : str
        Level of the task. Either "graph" or "node".
    """

    def __init__(
        self, in_channels, hidden_channels, out_channels, task_level="graph", **kwargs
    ):
        super().__init__()

        # Define the model
        self.base_model = HMPNN(
            in_channels=in_channels, hidden_channels=hidden_channels, **kwargs
        )

        # Readout
        self.linear = torch.nn.Linear(hidden_channels, out_channels)
        self.out_pool = task_level == "graph"

    def forward(self, x_0, x_1, incidence_1):
        # Base model
        x_0, x_1 = self.base_model(x_0, x_1, incidence_1)

        # Pool over all nodes in the hypergraph
        x = torch.max(x_0, dim=0)[0] if self.out_pool is True else x_0

        return self.linear(x)

Initialize the model

In [9]:
torch.unique(y).shape[0]

7

In [10]:
# Base model hyperparameters
in_channels = x_0s.shape[1]
hidden_channels = 128
n_layers = 1

# Readout hyperparameters
out_channels = torch.unique(y).shape[0]
task_level = "graph" if out_channels == 1 else "node"


model = Network(
    in_channels=in_channels,
    hidden_channels=hidden_channels,
    out_channels=out_channels,
    n_layers=n_layers,
    task_level=task_level,
).to(device)

In [11]:
print(model)

Network(
  (base_model): HMPNN(
    (linear_node): Linear(in_features=1433, out_features=128, bias=True)
    (linear_edge): Linear(in_features=1433, out_features=128, bias=True)
    (layers): ModuleList(
      (0): HMPNNLayer(
        (node_to_hyperedge_messenger): _NodeToHyperedgeMessenger(
          (messaging_func): Sequential(
            (0): Linear(in_features=128, out_features=128, bias=True)
            (1): Sigmoid()
          )
        )
        (hyperedge_to_node_messenger): _HyperedgeToNodeMessenger(
          (messaging_func): _DefaultHyperedgeToNodeMessagingFunc(
            (linear): Linear(in_features=256, out_features=128, bias=True)
          )
        )
        (node_batchnorm): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (hyperedge_batchnorm): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (updating_func): _DefaultUpdatingFunc()
      )
    )
  )
  (linear): Linear(in_features=128

In [12]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.CrossEntropyLoss()


train_mask = dataset["train_mask"]
val_mask = dataset["val_mask"]
test_mask = dataset["test_mask"]

Now it's time to train the model, looping over the network for a low amount of epochs. We keep training minimal for the purpose of rapid testing.

**Note: The number of epochs below have been kept low to facilitate debugging and testing. Real use cases should likely require more epochs.**

Note that:
- x_0s: The input feature matrix of the node.
- initial_x_1:
- incidence_1: The adjacency matrix of the hypergraph

In [13]:
torch.manual_seed(0)
test_interval = 5
num_epochs = 1000


initial_x_1 = torch.zeros_like(x_0s)
for epoch in range(1, num_epochs + 1):
    model.train()
    optimizer.zero_grad()
    y_hat = model(x_0s, initial_x_1, incidence_1)
    loss = loss_fn(y_hat[train_mask], y[train_mask])
    loss.backward()
    optimizer.step()

    train_loss = loss.item()
    y_pred = y_hat.argmax(dim=-1)
    train_acc = accuracy_score(y[train_mask].cpu(), y_pred[train_mask].cpu())

    if epoch % test_interval == 0:
        model.eval()

        y_hat = model(x_0s, initial_x_1, incidence_1)
        val_loss = loss_fn(y_hat[val_mask], y[val_mask]).item()
        y_pred = y_hat.argmax(dim=-1)
        val_acc = accuracy_score(y[val_mask].cpu(), y_pred[val_mask].cpu())

        test_loss = loss_fn(y_hat[test_mask], y[test_mask]).item()
        y_pred = y_hat.argmax(dim=-1)
        test_acc = accuracy_score(y[test_mask].cpu(), y_pred[test_mask].cpu())
        print(
            f"Epoch: {epoch + 1} train loss: {train_loss:.4f} train acc: {train_acc:.2f} "
            f" val loss: {val_loss:.4f} val acc: {val_acc:.2f}"
            f" test loss: {test_acc:.4f} val acc: {test_acc:.2f}"
        )

Epoch: 6 train loss: 1.2665 train acc: 0.82  val loss: 1.9848 val acc: 0.23 test loss: 0.2270 val acc: 0.23
Epoch: 11 train loss: 0.8439 train acc: 0.99  val loss: 1.7568 val acc: 0.38 test loss: 0.3970 val acc: 0.40
Epoch: 16 train loss: 0.5118 train acc: 1.00  val loss: 1.6843 val acc: 0.40 test loss: 0.4150 val acc: 0.41
Epoch: 21 train loss: 0.2717 train acc: 1.00  val loss: 1.5870 val acc: 0.43 test loss: 0.4500 val acc: 0.45
Epoch: 26 train loss: 0.1571 train acc: 1.00  val loss: 1.6138 val acc: 0.41 test loss: 0.4220 val acc: 0.42
Epoch: 31 train loss: 0.0816 train acc: 1.00  val loss: 1.5894 val acc: 0.45 test loss: 0.4490 val acc: 0.45
Epoch: 36 train loss: 0.0478 train acc: 1.00  val loss: 1.6026 val acc: 0.46 test loss: 0.4620 val acc: 0.46
Epoch: 41 train loss: 0.0298 train acc: 1.00  val loss: 1.6143 val acc: 0.47 test loss: 0.4670 val acc: 0.47
Epoch: 46 train loss: 0.0214 train acc: 1.00  val loss: 1.6487 val acc: 0.47 test loss: 0.4730 val acc: 0.47
Epoch: 51 train loss

KeyboardInterrupt: 

In [22]:
print(y_hat[train_mask])

tensor([[-2.3404, -2.3616, -1.0914, 13.5829, -1.2837, -2.1233, -1.9325],
        [-3.2887, -2.6952, -2.7585, -3.0699, 12.0892, -1.7336, -2.0955],
        [-2.7816, -3.1118, -2.4088, -3.2886, 11.7967, -2.2970, -1.7538],
        [11.3011, -2.4660, -2.1042, -3.9745, -2.9593, -2.6895, -2.0052],
        [-3.0607, -2.2961, -0.9514, 14.2813, -1.5308, -1.7687, -1.6491],
        [-1.7914, -2.5848, 13.5673, -2.3254, -1.9857, -2.4514, -1.8735],
        [11.6065, -2.2457, -2.4827, -4.0519, -3.0297, -2.2605, -2.6045],
        [-2.7894, -2.1635, -1.1661, 14.3403, -1.4402, -1.7256, -2.1138],
        [-2.4734, -2.5382, -1.2406, 14.4546, -1.3764, -1.7861, -2.2584],
        [-1.7141, -2.4663, 11.6857, -2.3136, -1.2823, -2.0484, -1.6935],
        [11.5254, -2.1919, -2.1277, -3.6737, -3.4718, -2.4170, -2.3764],
        [11.7715, -2.3391, -2.1153, -3.8919, -3.3441, -2.6664, -2.2455],
        [-3.0241, -2.9473, -2.3750, -3.3107, 11.6597, -2.1437, -1.5873],
        [-2.5554, -2.3240, -1.4496, 14.2534, -1.398

In [15]:
print(train_mask)

tensor([ True,  True,  True,  ..., False, False, False], device='cuda:0')
